<center><img src='https://i.postimg.cc/TPR1n1rp/AI-Tech-PL-RGB.png' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://i.postimg.cc/Gpq2KRQz/logotypy-aitech.jpg'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
</center>

# Lab 09: Imitation Learning

In this lab, we look into the problem of learning from expert demonstrations.

- Find a policy $\pi(a | s)$ that best imitates the expert policy $\pi^*(a | s)$ in the given environment.
- It's worth noting, that we don't need access to the environment rewards.

Major Imitation Learning techniques are:

1. Behavioural Cloning,
1. Imitation Learning via Interactive Demonstrator e.g. SMILe (Ross and Bagnell, 2010) or DAgger (Ross et al., 2011),
1. Inverse Reinforcement Learning -- out of scope of this lab.

We will solve the Ant problem, shown below, examining the first two approaches.

In [1]:
#@title Mount your Google Drive

#@markdown Your work will be stored in a folder called `rl_lab_2022` by default.

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

import os
from google.colab import drive
drive.mount('/content/gdrive')

LAB_PATH = '/content/gdrive/MyDrive/rl_lab_2022/imitation_learning'
if not os.path.exists(LAB_PATH):
  %mkdir -p $LAB_PATH

MJC_PATH = '{}/mujoco'.format(LAB_PATH)
if not os.path.exists(MJC_PATH):
    %mkdir $MJC_PATH

Mounted at /content/gdrive


In [2]:
#@title Install requirements

!apt -q update 
!apt install -q -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg
!pip -q install gdown

# Installing dependencies for visualization
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install -U gym==0.19
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [3]:
#@title Download MuJoCo

if not os.path.exists(f'{MJC_PATH}/mujoco210'):
    %cd $MJC_PATH
    !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
    !tar -xzf mujoco210-linux-x86_64.tar.gz
    %rm mujoco210-linux-x86_64.tar.gz

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco


In [4]:
#@title Install MuJoCo

import os

os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco210/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco210'.format(MJC_PATH)

# Installation on colab does not find *.so files in LD_LIBRARY_PATH,
# copy over manually instead.
!cp $MJC_PATH/mujoco210/bin/*.so /usr/lib/x86_64-linux-gnu/

In [5]:
#@title Clone and install mujoco-py

if not os.path.exists(f'{MJC_PATH}/mujoco-py'):
    %cd $MJC_PATH
    !git clone https://github.com/openai/mujoco-py.git

%cd $MJC_PATH/mujoco-py
!git checkout f1312cceeeebbba17e78d5d77fbffa091eed9a3a # Tested version
%pip install -e .

# Compile at the first import
os.environ['LD_LIBRARY_PATH'] += ':/usr/lib/nvidia'
os.environ['LD_LIBRARY_PATH'] += ':/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco210/bin'
import mujoco_py

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco
Cloning into 'mujoco-py'...
remote: Enumerating objects: 2183, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2183 (delta 7), reused 20 (delta 1), pack-reused 2148
Receiving objects: 100% (2183/2183), 5.70 MiB | 7.66 MiB/s, done.
Resolving deltas: 100% (1350/1350), done.
Checking out files: 100% (200/200), done.
/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco-py
M	.dockerignore
M	.gitignore
M	mujoco_py/tests/test_substep.py
M	scripts/gen_wrappers.py
M	vendor/Xdummy-entrypoint
Note: checking out 'f1312cceeeebbba17e78d5d77fbffa091eed9a3a'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or 

In [6]:
#@title Download the expert checkpoint

if not os.path.exists(f'{LAB_PATH}/expert_checkpoint'):
    %cd $LAB_PATH
    !gdown --id 1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
    !unzip expert_checkpoint.zip

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_checkpoint.zip
100% 382k/382k [00:00<00:00, 71.9MB/s]
Archive:  expert_checkpoint.zip
   creating: expert_checkpoint/
  inflating: __MACOSX/._expert_checkpoint  
   creating: expert_checkpoint/variables/
  inflating: __MACOSX/expert_checkpoint/._variables  
  inflating: expert_checkpoint/saved_model.pb  
   creating: expert_checkpoint/assets/
  inflating: __MACOSX/expert_checkpoint/._assets  
  inflating: expert_checkpoint/variables/variables.data-00000-of-00001  
  inflating: expert_checkpoint/variables/variables.index  


In [7]:
import glob
import time

from functools import partial

import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from base64 import b64encode
from IPython.display import HTML
from pyvirtualdisplay import Display

# Start virtual display
display = Display(visible=0, size=(1024, 768))
display.start()

# Seed random generators
tf.random.set_seed(42)
np.random.seed(42)

# Helpers

def show_video(file_name):
    mp4 = open(file_name,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=480 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

def mlp(input_shape, output_size, hidden_sizes=(256, 256), hidden_activation=tf.tanh, output_activation=None, l2_weight=0.0001):
    """Creates MLP with the specified parameters."""
    model = tf.keras.Sequential()

    model.add(tf.keras.Input(shape=input_shape))
    for h in hidden_sizes:
        model.add(tf.keras.layers.Dense(units=h,
                                        activation=hidden_activation,
                                        kernel_regularizer=tf.keras.regularizers.L2(l2_weight)))
    model.add(tf.keras.layers.Dense(units=output_size, activation=output_activation))

    return model

def run_policy (env, model, total_steps=10000, verbose=True):
    obs_array = np.empty([total_steps, *env.observation_space.shape])
    act_array = np.empty([total_steps, env.action_space.shape[0]])
    rew_array = np.empty([total_steps, 1])
    done_array = np.empty([total_steps, 1])

    iter_time = time.time()
    done = True
    for i in range(total_steps):
        if verbose and (i + 1) % 1000 == 0:
            steps_per_second = 1000 / (time.time() - iter_time)
            print(f'Step {i + 1}/{total_steps}, Steps per second: {steps_per_second}')
            iter_time = time.time()


        if done:
            obs = env.reset()

        act = model(tf.expand_dims(obs, axis=0))[0]
        obs_, rew, done, _ = env.step(act)
        
        obs_array[i] = obs
        act_array[i] = act
        rew_array[i] = rew
        done_array[i] = float(done)

        obs = obs_

    return obs_array, act_array, rew_array, done_array

def calculate_returns(rew, done):
    rew_cumsum = np.cumsum(rew)[:, None]
    ret_cumsum = rew_cumsum * done
    ret_cumsum_trimed = ret_cumsum[np.nonzero(ret_cumsum)]
    ret_cumsum_trimed[1:] -= ret_cumsum_trimed[:-1]
    return ret_cumsum_trimed

def evaluate_agent(env, model, verbose=False):
    _, _, rew, done = run_policy(env, model, total_steps=25000, verbose=verbose)
    rets = calculate_returns(rew, done)

    print(f'Num. episodes: {len(rets)}')
    print(f'Avg. return: {np.mean(rets)}')
    print(f'Max. return: {np.max(rets)}')
    print(f'Min. return: {np.min(rets)}')

def render_agent(env, model):
    envw = gym.wrappers.Monitor(env, "./", force=True)
    o, d = envw.reset(), False
    while not d:
        envw.render()
        o, _, d, _ = envw.step(model(tf.expand_dims(o, axis=0))[0])
    # envw.close()

    file_name = glob.glob('openaigym.video.*.mp4')[0]
    return show_video(file_name)

class Expert:
    """Streamlined Off-Policy (SOP) actor"""

    def __init__(self, ckpt_path):
        self.model = tf.keras.models.load_model(ckpt_path)

    def __call__(self, obs, exploratory=False):
        # We need to add one more dim. for this model
        mu, pi = self.model(tf.expand_dims(obs, axis=0))
        return pi[0] if exploratory else mu[0]

## 0. Ant

a three-dimensional quadrupedal robot.

- Observations are 111-dim. vectors that describe the kinematic properties of the robot,
- Actions are 8-dim. vectors which specify torques to be applied on the robot joints,
- The goal is to run forward as fast as possible and don’t fall over.

In [8]:
%cd $LAB_PATH
expert = Expert('expert_checkpoint')
env = gym.make('Ant-v2')

/content/gdrive/MyDrive/rl_lab_2022/imitation_learning


In [16]:
# render_agent(env, expert)

In [10]:
evaluate_agent(env, expert, verbose=True)

Step 1000/25000, Steps per second: 117.79707573805796
Step 2000/25000, Steps per second: 116.68760529708865
Step 3000/25000, Steps per second: 117.35284379275032
Step 4000/25000, Steps per second: 118.51199506501412
Step 5000/25000, Steps per second: 116.66140050288156
Step 6000/25000, Steps per second: 116.18754091131079
Step 7000/25000, Steps per second: 114.78175696732782
Step 8000/25000, Steps per second: 117.88899995185284
Step 9000/25000, Steps per second: 109.95318808365798
Step 10000/25000, Steps per second: 78.93528868884532
Step 11000/25000, Steps per second: 116.08535203140731
Step 12000/25000, Steps per second: 116.18616982848907
Step 13000/25000, Steps per second: 116.96328542062389
Step 14000/25000, Steps per second: 116.72504722374264
Step 15000/25000, Steps per second: 104.53064903594537
Step 16000/25000, Steps per second: 116.63235984720656
Step 17000/25000, Steps per second: 112.7373427503316
Step 18000/25000, Steps per second: 112.81579543357772
Step 19000/25000, Ste

## 1. Behaviour Clonning

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.

In [17]:
# Collect the expert data
# obs, act, _, _ = run_policy(env, expert, total_steps=100000)

# Download the expert data instead of collecting it
if not os.path.exists(f'expert_greedy_data.npz'):
    !gdown --id 1Ny7xe3aZR-u2TpaoNhDG6wI4SS5N8FR8
with np.load('expert_greedy_data.npz') as data:
    obs, act, _, _ = data.values()

Exception ignored in: <function Monitor.__del__ at 0x7fc7b8d13cb0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitor.py", line 287, in __del__
    self.close()
  File "/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitor.py", line 182, in close
    self._close_video_recorder()
  File "/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitor.py", line 268, in _close_video_recorder
    self.video_recorder.close()
  File "/usr/local/lib/python3.7/dist-packages/gym/wrappers/monitoring/video_recorder.py", line 157, in close
    os.remove(self.path)
FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/openaigym.video.4.59.video000000.mp4'


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Ny7xe3aZR-u2TpaoNhDG6wI4SS5N8FR8
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_greedy_data.npz
100% 96.8M/96.8M [00:00<00:00, 132MB/s]


In [19]:
# EXERCISE: Create the imitator model
imitator = mlp(111, 8)

# We will start our experiments from the same weights for the fair comparison
init_weights = imitator.get_weights()

In [21]:
# EXERCISE: Fit the model to the expert data
imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs, act, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 11s 3ms/step - loss: 0.1020
Epoch 2/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0656
Epoch 3/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0595
Epoch 4/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.0563
Epoch 5/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0542
Epoch 6/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0525
Epoch 7/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0512
Epoch 8/25
3125/3125 [==============================] - 15s 5ms/step - loss: 0.0503
Epoch 9/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0497
Epoch 10/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0491
Epoch 11/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0488
Epoch 12/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.0484

In [22]:
# Collect the exploratory data
def exploratory(obs):
    """Adds the Gaussian noise to the expert actions."""
    act = expert(obs)
    return act + 0.29 * tf.random.normal(tf.shape(act))
# obs_expl, act_expl, rew_expl, done_expl = run_policy(env, exploratory, total_steps=100000)

# Download instead...
if not os.path.exists(f'expert_exploratory_data.npz'):
    !gdown --id 1O-xDKmH0a8i6Q5Zfmqc6qZ0Yi8H3q5ZB
with np.load('expert_exploratory_data.npz') as data:
    obs_expl, act_expl, rew_expl, done_expl = data.values()

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1O-xDKmH0a8i6Q5Zfmqc6qZ0Yi8H3q5ZB
To: /content/gdrive/MyDrive/rl_lab_2022/imitation_learning/expert_exploratory_data.npz
100% 96.8M/96.8M [00:01<00:00, 90.4MB/s]


In [23]:
# EXERCISE: Run BC on the exploratory data

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_expl, act_expl, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1759
Epoch 2/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.1359
Epoch 3/25
3125/3125 [==============================] - 11s 4ms/step - loss: 0.1278
Epoch 4/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.1241
Epoch 5/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.1218
Epoch 6/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.1202
Epoch 7/25
3125/3125 [==============================] - 8s 3ms/step - loss: 0.1190
Epoch 8/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1180
Epoch 9/25
3125/3125 [==============================] - 9s 3ms/step - loss: 0.1171
Epoch 10/25
3125/3125 [==============================] - 10s 3ms/step - loss: 0.1165
Epoch 11/25
3125/3125 [==============================] - 10s 3ms/step - loss: 0.1160
Epoch 12/25
3125/3125 [==============================] - 11s 4ms/step - loss: 0.1

In [37]:
# EXERCISE: Infere the expert actions on the exploratory observations
#           and run BC on it.

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_expl, exploratory(obs_expl), epochs=25)
# ANSWER END

evaluate_agent(env, imitator)

Epoch 1/25
3125/3125 [==============================] - 14s 4ms/step - loss: 0.2239
Epoch 2/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1820
Epoch 3/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1733
Epoch 4/25
3125/3125 [==============================] - 11s 4ms/step - loss: 0.1687
Epoch 5/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1663
Epoch 6/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1645
Epoch 7/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1630
Epoch 8/25
3125/3125 [==============================] - 12s 4ms/step - loss: 0.1617
Epoch 9/25
3125/3125 [==============================] - 11s 3ms/step - loss: 0.1607
Epoch 10/25
3125/3125 [==============================] - 10s 3ms/step - loss: 0.1598
Epoch 11/25
3125/3125 [==============================] - 10s 3ms/step - loss: 0.1592
Epoch 12/25
3125/3125 [==============================] - 12s 4ms/step - lo

## 2. Imitation Learning via Interactive Demostrator

[DAgger](https://www.ri.cmu.edu/pub_files/2011/4/Ross-AISTATS11-NoRegret.pdf)

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.
3. Collect the imitator data.
4. Infere the expert actions on the imitator data.
5. Fit the model to the extended dataset.
6. Repeat from 3.

In [34]:
# We will pre-train on less expert data to keep the same dataset size
obs_ = obs[:30000,:]
act_ = act[:30000,:]

In [35]:
# EXERCISE: Pretrain for 25 epochs

imitator.set_weights(init_weights)

# ANSWER

imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_, act_, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

Epoch 1/25
938/938 [==============================] - 3s 3ms/step - loss: 0.1408
Epoch 2/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0930
Epoch 3/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0780
Epoch 4/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0705
Epoch 5/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0662
Epoch 6/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0635
Epoch 7/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0613
Epoch 8/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0600
Epoch 9/25
938/938 [==============================] - 2s 3ms/step - loss: 0.0586
Epoch 10/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0575
Epoch 11/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0566
Epoch 12/25
938/938 [==============================] - 3s 3ms/step - loss: 0.0556
Epoch 13/25
938/938 [====

In [36]:
# Exercise: Implement DAgger

for i in range(7):
    print(f'\n### Iter. {i+1} ###')

    # ANSWER
    print('\n1. Data collection')
    obs_extra, _, _, _ = run_policy(env, imitator) # Collect 10k steps
    act_extra = expert(obs_extra)

    obs_ = np.vstack([obs_, obs_extra])
    act_ = np.vstack([act_, act_extra])

    print('\n2. Training')
    imitator.set_weights(init_weights)
    

    imitator.compile(loss=tf.keras.losses.MeanSquaredError())
    imitator.fit(obs_, act_, epochs=5)
    # END ANSWER

    print('\n3. Evaluation')
    evaluate_agent(env, imitator)


### Iter. 1 ###

1. Data collection
Step 1000/10000, Steps per second: 415.3034465905405
Step 2000/10000, Steps per second: 403.73667110995086
Step 3000/10000, Steps per second: 407.4520433394968
Step 4000/10000, Steps per second: 404.9006221541237
Step 5000/10000, Steps per second: 419.4054453760001
Step 6000/10000, Steps per second: 419.2715380142464
Step 7000/10000, Steps per second: 323.2699413578111
Step 8000/10000, Steps per second: 358.0534198495243
Step 9000/10000, Steps per second: 403.3530305494214
Step 10000/10000, Steps per second: 409.95983584028636

2. Training
Epoch 1/5
1250/1250 [==============================] - 4s 3ms/step - loss: 0.1393
Epoch 2/5
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0962
Epoch 3/5
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0831
Epoch 4/5
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0766
Epoch 5/5
1250/1250 [==============================] - 3s 3ms/step - loss: 0.0720

3. Eva

### Note

Training the expert with the SOP algorithm (Wang et al., 2020) took 3M data samples (env. interactions). Here, we nearly match it with only 100k samples! Training from the expert can be much more efficient than reinforcement learning.